In this tutorial, we show how to run the [FAST tutorial example](https://web.stanford.edu/~lcambier/fast/tuto.php) using this package.
The big difference between this example and the quickstart example is that in this example we will model serial independence.
There will be 5 stages and 2 scenarios per stages except for the first stage which has only one scenario.
Each pair of scenario will have the same parent.

We start by setting the constants:

In [1]:
const num_stages = 5
const numScen = 2
const C = 5
const V = 8
const d = 6
const r = [2, 10];

We now create a matrix to store all the variables of all the models.
This allows us to use the variables of other models from a given model.
We also create an array of the first model of each stage to give play the role of parent for the models of the next stage.

In [2]:
using StructJuMP
x = Matrix{JuMP.Variable}(undef, num_stages, numScen)
y = Matrix{JuMP.Variable}(undef, num_stages, numScen)
p = Matrix{JuMP.Variable}(undef, num_stages, numScen)
models = Vector{JuMP.Model}(undef, num_stages);

Now, we create all the models.
Note that each model declares that its parent is the first model (i.e. the model `ξ == 1`) of the previous stage.
Hence if it is not the first model, it also declares that it has the same children than the first model of its stage.
This is how serial independence is modeled in [StructJuMP](https://github.com/StructJuMP/StructJuMP.jl).

In [3]:
using Statistics
for s in 1:num_stages
    for ξ in 1:(s == 1 ? 1 : numScen) # for the first stage there is only 1 scenario
        if s == 1
            model = StructuredModel(num_scenarios=numScen)
        else
            model = StructuredModel(parent=models[s-1], prob=1/2, same_children_as=(ξ == 1 ? nothing : models[s]), id=ξ, num_scenarios=(s == num_stages ? 0 : numScen))
        end
        x[s, ξ] = @variable(model, lowerbound=0, upperbound=V)
        y[s, ξ] = @variable(model, lowerbound=0)
        p[s, ξ] = @variable(model, lowerbound=0)
        if s > 1
            @constraint(model, x[s, ξ] <= x[s-1, 1] + r[ξ] - y[s, ξ])
        else
            @constraint(model, x[s, ξ] <= mean(r) - y[s, ξ])
        end
        @constraint(model, p[s, ξ] + y[s, ξ] >= d)
        @objective(model, Min, C * p[s, ξ])
        # models[s] contains the first model only
        if ξ == 1
            models[s] = model
        end
    end
end

We first need to pick an LP solver, see [here](http://www.juliaopt.org/) for a list of the available choices.

In [4]:
using GLPKMathProgInterface
solver = GLPKMathProgInterface.GLPKSolverLP();

We now create the lattice, note that the master problem is `models[1]`.

In [5]:
using CutPruners
const pruner = AvgCutPruningAlgo(-1)
using StructDualDynProg
const SOI = StructDualDynProg.SOI
sp = SOI.stochasticprogram(models[1], num_stages, solver, pruner);

The SDDP algorithm can now be run on the lattice:

In [6]:
algo = StructDualDynProg.SDDP.Algorithm(K = 16)
sol = SOI.optimize!(sp, algo, SOI.Pereira(2., 0.5) | SOI.IterLimit(10))

Exact Lower Bound: 23.75
Monte-Carlo Upper Bound: 24.375

 ──────────────────────────────────────────────────────────────────────
                               Time                   Allocations      
                       ──────────────────────   ───────────────────────
   Tot / % measured:        6.08s / 90.7%            946MiB / 91.9%    

 Section       ncalls     time   %tot     avg     alloc   %tot      avg
 ──────────────────────────────────────────────────────────────────────
 

iteration 1        1    5.51s   100%   5.51s    868MiB  100%    868MiB
   solve           25    1.84s  33.4%  73.6ms    231MiB  26.6%  9.25MiB
   ocuts           10    184ms  3.33%  18.4ms   29.1MiB  3.35%  2.91MiB
   merged           4    144ms  2.62%  36.1ms   21.7MiB  2.50%  5.43MiB
   setx            24   32.8ms  0.60%  1.37ms   4.03MiB  0.46%   172KiB
 iteration 2        1   1.52ms  0.03%  1.52ms    810KiB  0.09%   810KiB
   solve           41    850μs  0.02%  20.7μs    350KiB  0.04%  8.53KiB
   setx            40    438μs  0.01%  10.9μs    244KiB  0.03%  6.11KiB
   merged           4   24.0μs  0.00%  6.00μs   9.94KiB  0.00%  2.48KiB
   ocuts            1   8.51μs  0.00%  8.51μs   1.95KiB  0.00%  1.95KiB
 ──────────────────────────────────────────────────────────────────────
